# Exercise 2.1 - Inspection

## Objective

- To see how to inspect the outputs of various stages of Numba’s compilation pipeline.

## Typing

First, we need a function to work with, so we’ll invent this sum function:

In [ ]:
from numba import jit

@jit
def sum(x):
    s = 0
    for i in range(len(x)):
        s += x[i]
    return s

Now inspect its types:



In [ ]:
sum.inspect_types()

There is no output - no typing information exists at present, because we have not yet forced a compilation by calling the function yet. So let’s do that and try inspecting types again:

In [ ]:
import numpy as np

a = np.arange(10)

sum(a)

sum.inspect_types()

Calling the function again with arguments of a different type results in a different typing:

In [ ]:
a = np.arange(10, dtype=np.float32)
sum(a)
sum.inspect_types()

The different typings have different code generated.

## LLVM IR

We can use inspect_llvm to see the generated LLVM IR, which return a dict keyed on the argument types, so it it helpful to define an additional function to help show the LLVM code:

In [ ]:
def show_llvm(func):
    llvm_code = func.inspect_llvm()
    for k, v in llvm_code.items():
        print('-' * 80)
        print("Signature:", k)
        print('-' * 80)
        print(v)
        print()

Now we can easily look at the LLVM code for sum:

In [ ]:
show_llvm(sum)

That is quite a lot of code! The generated code includes a wrapper function which does part of the work of marshalling the arguments, and calls to the Numba Runtime, which manages memory allocation within compiled code.

## Assembly

Generated assembly can similarly be viewed:

In [ ]:
def show_asm(func):
    asm_code = func.inspect_asm()
    for k, v in asm_code.items():
        print('-' * 80)
        print("Signature:", k)
        print('-' * 80)
        print(v)
        print()

show_asm(sum)

The generated assembly code is a lot shorter than the LLVM code - this is because it has been transformed by LLVM’s optimisation passes, which has in part simplified the code so that it executes more quickly.

## Summary

- The typing of variables in the Python source and Numba IR can be viewed with `inspect_types()`.
- The generated LLVM and Assembly code can be retrieved using the `inspect_llvm()` and `inspect_asm()` functions.
- The LLVM output is very large in comparison with the assembly code. This is because the optimisation passes simplify and eliminate a large amount of code.
- The generated code also handles marshalling Python arguments to native types, and book-keeping for reference-counting.